<a href="https://colab.research.google.com/github/WITwilliamsw5atwit/DATA-6150-Individual-Project-Wayne-Williams/blob/main/FinalProject_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets evaluate sacrebleu rouge-score nltk

import os
import zipfile
import glob
import random
import numpy as np
import torch
import nltk

from google.colab import files
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
)
import evaluate

nltk.download('punkt')


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.2 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Upload your zip files from your computer into Colab
print("Please upload train.zip, validation.zip, test.zip (and/or the DailyDialog zip).")
uploaded = files.upload()

DATA_ROOT = "dailydialog_local"
os.makedirs(DATA_ROOT, exist_ok=True)

# Unzip everything into DATA_ROOT
for fname in uploaded.keys():
    if fname.lower().endswith(".zip"):
        print(f"Unzipping {fname} ...")
        with zipfile.ZipFile(fname, "r") as z:
            z.extractall(DATA_ROOT)
    else:
        print(f"Skipping non-zip file: {fname}")

print("\nDirectory structure under dailydialog_local:")
for root, dirs, files_in_dir in os.walk(DATA_ROOT):
    print(root, "->", files_in_dir)


Please upload train.zip, validation.zip, test.zip (and/or the DailyDialog zip).


Saving test.zip to test (1).zip
Saving train.zip to train (1).zip
Saving validation.zip to validation (1).zip
Unzipping test (1).zip ...
Unzipping train (1).zip ...
Unzipping validation (1).zip ...

Directory structure under dailydialog_local:
dailydialog_local -> []
dailydialog_local/train -> ['dialogues_act_train.txt', 'dialogues_emotion_train.txt', 'dialogues_train.txt']
dailydialog_local/validation -> ['dialogues_validation.txt', 'dialogues_act_validation.txt', 'dialogues_emotion_validation.txt']
dailydialog_local/test -> ['dialogues_act_test.txt', 'dialogues_test.txt', 'dialogues_emotion_test.txt']


In [ ]:
def find_file(patterns):
    """
    Try a list of glob patterns and return the first existing file found.
    """
    for pat in patterns:
        matches = glob.glob(os.path.join(DATA_ROOT, "**", pat), recursive=True)
        if matches:
            print(f"Found: {matches[0]}")
            return matches[0]
    return None

# Common names in DailyDialog releases
train_path = find_file(["*train*.txt", "*Train*.txt"])
val_path   = find_file(["*validation*.txt", "*valid*.txt", "*Val*.txt"])
test_path  = find_file(["*test*.txt", "*Test*.txt"])

print("\nSelected files:")
print("Train:", train_path)
print("Val:  ", val_path)
print("Test: ", test_path)


Found: dailydialog_local/train/dialogues_act_train.txt
Found: dailydialog_local/validation/dialogues_validation.txt
Found: dailydialog_local/test/dialogues_act_test.txt

Selected files:
Train: dailydialog_local/train/dialogues_act_train.txt
Val:   dailydialog_local/validation/dialogues_validation.txt
Test:  dailydialog_local/test/dialogues_act_test.txt


In [ ]:
def load_dialogues_flexible(path, delimiter="__eou__"):
    """
    Tries to handle:
    - one-dialogue-per-line with `__eou__` delimiter, OR
    - one-utterance-per-line with blank lines between dialogues.
    """
    dialogues = []
    current = []

    with open(path, "r", encoding="utf-8") as f:
        for raw_line in f:
            line = raw_line.strip()

            if not line:
                # blank line => end of current dialogue if using line-by-line format
                if current:
                    dialogues.append(current)
                    current = []
                continue

            if delimiter in line:
                # flush any ongoing line-by-line dialogue
                if current:
                    dialogues.append(current)
                    current = []

                utts = [u.strip() for u in line.split(delimiter) if u.strip()]
                if len(utts) > 1:
                    dialogues.append(utts)
            else:
                # treat as a single utterance in a multi-line dialogue
                current.append(line)

        # flush last dialogue
        if current:
            dialogues.append(current)

    return dialogues


In [ ]:
train_dialogs = load_dialogues_flexible(train_path)
val_dialogs   = load_dialogues_flexible(val_path)
test_dialogs  = load_dialogues_flexible(test_path)


In [ ]:
if len(train_dialogs) > 0:
    print(train_dialogs[0][:3])
else:
    print("train_dialogs is empty")


['3 4 2 2 2 3 4 1 3 4', '2 1 2 2 1 1', '2 1 2 1 1']


In [ ]:
import os, glob

print("TXT files under dailydialog_local:")
for p in glob.glob("dailydialog_local/**/*.txt", recursive=True):
    print(p)


TXT files under dailydialog_local:
dailydialog_local/train/dialogues_act_train.txt
dailydialog_local/train/dialogues_emotion_train.txt
dailydialog_local/train/dialogues_train.txt
dailydialog_local/validation/dialogues_validation.txt
dailydialog_local/validation/dialogues_act_validation.txt
dailydialog_local/validation/dialogues_emotion_validation.txt
dailydialog_local/test/dialogues_act_test.txt
dailydialog_local/test/dialogues_test.txt
dailydialog_local/test/dialogues_emotion_test.txt


In [ ]:
train_path = "dailydialog_local/dialogues_text_train.txt"
val_path   = "dailydialog_local/dialogues_text_validation.txt"
test_path  = "dailydialog_local/dialogues_text_test.txt"


In [ ]:
import glob

print("All text files in your environment:\n")
for p in glob.glob("**/*.txt", recursive=True):
    print(p)



All text files in your environment:

dailydialog_local/train/dialogues_act_train.txt
dailydialog_local/train/dialogues_emotion_train.txt
dailydialog_local/train/dialogues_train.txt
dailydialog_local/validation/dialogues_validation.txt
dailydialog_local/validation/dialogues_act_validation.txt
dailydialog_local/validation/dialogues_emotion_validation.txt
dailydialog_local/test/dialogues_act_test.txt
dailydialog_local/test/dialogues_test.txt
dailydialog_local/test/dialogues_emotion_test.txt


In [ ]:
train_path = "dailydialog_local/train/dialogues_train.txt"
val_path   = "dailydialog_local/validation/dialogues_validation.txt"
test_path  = "dailydialog_local/test/dialogues_test.txt"


In [ ]:
train_dialogs = load_dialogues_flexible(train_path)
val_dialogs   = load_dialogues_flexible(val_path)
test_dialogs  = load_dialogues_flexible(test_path)

print("Train:", len(train_dialogs))
print("Val:  ", len(val_dialogs))
print("Test: ", len(test_dialogs))

print("\nExample train dialogue:")
print(train_dialogs[0][:3])



Train: 11118
Val:   1000
Test:  1000

Example train dialogue:
['Say , Jim , how about going for a few beers after dinner ?', 'You know that is tempting but is really not good for our fitness .', 'What do you mean ? It will help us to relax .']


In [ ]:
import os

print("=== Directory Tree ===")
for root, dirs, files in os.walk("dailydialog_local"):
    print(root, "->", files)


=== Directory Tree ===
dailydialog_local -> []
dailydialog_local/train -> ['dialogues_act_train.txt', 'dialogues_emotion_train.txt', 'dialogues_train.txt']
dailydialog_local/validation -> ['dialogues_validation.txt', 'dialogues_act_validation.txt', 'dialogues_emotion_validation.txt']
dailydialog_local/test -> ['dialogues_act_test.txt', 'dialogues_test.txt', 'dialogues_emotion_test.txt']


In [ ]:
import os

for name in ["train.zip", "validation.zip", "test.zip", "1dailydialog.zip"]:
    print(name, "exists?", os.path.exists(name))


train.zip exists? True
validation.zip exists? True
test.zip exists? True
1dailydialog.zip exists? True


In [ ]:
import os, zipfile

DATA_ROOT = "dailydialog_local"
os.makedirs(DATA_ROOT, exist_ok=True)

zip_names = ["train.zip", "validation.zip", "test.zip", "1dailydialog.zip"]

for fname in zip_names:
    if os.path.exists(fname):
        print(f"\nUnzipping {fname} ...")
        with zipfile.ZipFile(fname, "r") as z:
            z.extractall(DATA_ROOT)
    else:
        print(f"{fname} not found, skipping.")

print("\n=== Directory Tree under dailydialog_local ===")
for root, dirs, files_in_dir in os.walk(DATA_ROOT):
    print(root, "->", files_in_dir)



Unzipping train.zip ...

Unzipping validation.zip ...

Unzipping test.zip ...

Unzipping 1dailydialog.zip ...

=== Directory Tree under dailydialog_local ===
dailydialog_local -> []
dailydialog_local/train -> ['dialogues_act_train.txt', 'dialogues_emotion_train.txt', 'dialogues_train.txt']
dailydialog_local/1dailydialog -> ['README.md', 'dailydialog.py', 'gitattributes']
dailydialog_local/1dailydialog/train -> []
dailydialog_local/1dailydialog/train/train -> ['dialogues_act_train.txt', 'dialogues_emotion_train.txt', 'dialogues_train.txt']
dailydialog_local/1dailydialog/validation -> []
dailydialog_local/1dailydialog/validation/validation -> ['dialogues_validation.txt', 'dialogues_act_validation.txt', 'dialogues_emotion_validation.txt']
dailydialog_local/1dailydialog/test -> []
dailydialog_local/1dailydialog/test/test -> ['dialogues_act_test.txt', 'dialogues_test.txt', 'dialogues_emotion_test.txt']
dailydialog_local/validation -> ['dialogues_validation.txt', 'dialogues_act_validation.t

In [ ]:
train_path = "dailydialog_local/train/dialogues_train.txt"
val_path   = "dailydialog_local/validation/dialogues_validation.txt"
test_path  = "dailydialog_local/test/dialogues_test.txt"

print("Train path:", train_path)
print("Val path:  ", val_path)
print("Test path: ", test_path)


Train path: dailydialog_local/train/dialogues_train.txt
Val path:   dailydialog_local/validation/dialogues_validation.txt
Test path:  dailydialog_local/test/dialogues_test.txt


In [ ]:
def load_dialogues_txt(path, delimiter="__eou__"):
    dialogues = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            utts = [u.strip() for u in line.split(delimiter) if u.strip()]
            if len(utts) > 1:
                dialogues.append(utts)
    return dialogues

train_dialogs = load_dialogues_txt(train_path)
val_dialogs   = load_dialogues_txt(val_path)
test_dialogs  = load_dialogues_txt(test_path)

print("Loaded dialogues:")
print("Train:", len(train_dialogs))
print("Val:  ", len(val_dialogs))
print("Test: ", len(test_dialogs))
print("Example:", train_dialogs[0][:3])


Loaded dialogues:
Train: 11118
Val:   1000
Test:  1000
Example: ['Say , Jim , how about going for a few beers after dinner ?', 'You know that is tempting but is really not good for our fitness .', 'What do you mean ? It will help us to relax .']


In [ ]:
def make_context_response_pairs(dialogues, max_turns=None):
    contexts = []
    responses = []
    for dialog in dialogues:
        if max_turns is not None:
            dialog = dialog[:max_turns]
        for i in range(1, len(dialog)):
            ctx = " ".join(dialog[:i])
            resp = dialog[i]
            contexts.append(ctx)
            responses.append(resp)
    return contexts, responses

train_ctx, train_resp = make_context_response_pairs(train_dialogs)
val_ctx,   val_resp   = make_context_response_pairs(val_dialogs)
test_ctx,  test_resp  = make_context_response_pairs(test_dialogs)

print("Num train pairs:", len(train_ctx))
print("Num val pairs:  ", len(val_ctx))
print("Num test pairs: ", len(test_ctx))
print("\nExample pair:")
print("CONTEXT:", train_ctx[0])
print("RESPONSE:", train_resp[0])


Num train pairs: 76052
Num val pairs:   7069
Num test pairs:  6740

Example pair:
CONTEXT: Say , Jim , how about going for a few beers after dinner ?
RESPONSE: You know that is tempting but is really not good for our fitness .


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

MODEL_NAME = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Add a pad token if missing (GPT-2 style models usually don’t have one)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_batch(batch):
    # For causal LM we glue context + <eos> + response and predict all tokens
    texts = [
        c + tokenizer.eos_token + " " + r
        for c, r in zip(batch["context"], batch["response"])
    ]
    enc = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=128,
    )
    enc["labels"] = enc["input_ids"].copy()
    return enc

train_ds = Dataset.from_dict({"context": train_ctx, "response": train_resp})
val_ds   = Dataset.from_dict({"context": val_ctx,   "response": val_resp})
test_ds  = Dataset.from_dict({"context": test_ctx,  "response": test_resp})

# Optional: limit training size so Colab doesn’t die
MAX_TRAIN_SAMPLES = 20000  # or None to use all
if MAX_TRAIN_SAMPLES is not None:
    max_n = min(MAX_TRAIN_SAMPLES, len(train_ds))
    train_ds = train_ds.select(range(max_n))
    print(f"Using {max_n} training samples.")

train_ds = train_ds.map(tokenize_batch, batched=True, remove_columns=["context", "response"])
val_ds   = val_ds.map(tokenize_batch,   batched=True, remove_columns=["context", "response"])

train_ds.set_format(type="torch")
val_ds.set_format(type="torch")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Using 20000 training samples.


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7069 [00:00<?, ? examples/s]

In [ ]:
# Takes too long because it takes a larger batch
!pip install --upgrade transformers
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments
import torch

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model.config.pad_token_id = tokenizer.pad_token_id

training_args = TrainingArguments(
    output_dir="dialoGPT_finetuned_local",
    eval_strategy="epoch",   # <--- now works
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,
    fp16=torch.cuda.is_available(),
    report_to="none",
    gradient_accumulation_steps=4,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
)

trainer.train()

FT_MODEL_DIR = "dialoGPT_finetuned_local"
model.save_pretrained(FT_MODEL_DIR)
tokenizer.save_pretrained(FT_MODEL_DIR)

print("Fine-tuning complete; model saved at", FT_MODEL_DIR)


config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

MODEL_NAME = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Add a pad token if missing (GPT-2 style models usually don’t have one)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_batch(batch):
    # For causal LM we glue context + <eos> + response and predict all tokens
    texts = [
        c + tokenizer.eos_token + " " + r
        for c, r in zip(batch["context"], batch["response"])
    ]
    enc = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=128,
    )
    enc["labels"] = enc["input_ids"].copy()
    return enc

# Build full datasets from your pairs
full_train_ds = Dataset.from_dict({"context": train_ctx, "response": train_resp})
val_ds        = Dataset.from_dict({"context": val_ctx,   "response": val_resp})
test_ds       = Dataset.from_dict({"context": test_ctx,  "response": test_resp})

# 🔹 FAST MODE: only use a subset of training examples
MAX_TRAIN_SAMPLES = 10000   # you can lower this (e.g., 5000) for even faster runs

if MAX_TRAIN_SAMPLES is not None:
    max_n = min(MAX_TRAIN_SAMPLES, len(full_train_ds))
    train_ds = full_train_ds.select(range(max_n))
    print(f"Using {max_n} training samples out of {len(full_train_ds)} total.")
else:
    train_ds = full_train_ds

# Tokenize
train_ds = train_ds.map(tokenize_batch, batched=True, remove_columns=["context", "response"])
val_ds   = val_ds.map(tokenize_batch,   batched=True, remove_columns=["context", "response"])

train_ds.set_format(type="torch")
val_ds.set_format(type="torch")


Using 10000 training samples out of 76052 total.


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7069 [00:00<?, ? examples/s]

In [ ]:
#Also too long of time
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments
import torch

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model.config.pad_token_id = tokenizer.pad_token_id

training_args = TrainingArguments(
    output_dir="dialoGPT_finetuned_fast",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=5000,
    save_total_limit=1,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,   # still allowed, just not used during training
)

trainer.train()

model.save_pretrained("dialoGPT_finetuned_fast")
tokenizer.save_pretrained("dialoGPT_finetuned_fast")

print("Fast training complete.")


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments
import torch

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model.config.pad_token_id = tokenizer.pad_token_id

training_args = TrainingArguments(
    output_dir="dialoGPT_200steps",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,       # ignored once max_steps is set
    max_steps=200,            # 🔥 HARD LIMIT: only 200 update steps
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=20,         # print progress every 20 steps
    save_total_limit=1,       # keep only final checkpoint
    fp16=torch.cuda.is_available(),
    report_to="none"          # no wandb/tensorboard
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,      # fine to keep here
)

trainer.train()

model.save_pretrained("dialoGPT_200steps")
tokenizer.save_pretrained("dialoGPT_200steps")

print("✅ Training stopped at 200 steps and model saved to dialoGPT_200steps/")


Step,Training Loss
20,2.967300
40,2.340000
60,2.182100
80,2.180300
100,1.925100
120,2.006100
140,1.750500
160,1.852900
180,1.857500
200,1.937900


✅ Training stopped at 200 steps and model saved to dialoGPT_200steps/


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

BASE_MODEL_NAME = "microsoft/DialoGPT-small"

baseline_tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
baseline_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_NAME)


In [ ]:
ft_tokenizer = AutoTokenizer.from_pretrained("dialoGPT_200steps")
ft_model = AutoModelForCausalLM.from_pretrained("dialoGPT_200steps")



In [ ]:
import os
print(os.listdir("."))      # shows all files/folders in current directory


['.config', 'train.zip', 'dailydialog_local', 'test (1).zip', 'dialoGPT_finetuned_fast', 'dialoGPT_200steps', 'validation.zip', 'validation (1).zip', 'dialoGPT_finetuned_local', 'test.zip', '1dailydialog.zip', 'train (1).zip', 'sample_data']


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load baseline DialoGPT
BASE_MODEL_NAME = "microsoft/DialoGPT-small"
baseline_tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
baseline_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_NAME)

# Load fine-tuned 200-step model
ft_tokenizer = AutoTokenizer.from_pretrained("dialoGPT_200steps")
ft_model = AutoModelForCausalLM.from_pretrained("dialoGPT_200steps")

print("Models loaded successfully!")


Models loaded successfully!


In [ ]:
def chat(model, tokenizer, prompt):
    inputs = tokenizer.encode(prompt + tokenizer.eos_token, return_tensors="pt")
    outputs = model.generate(
        inputs,
        max_length=60,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = "I'm feeling stressed about my exam tomorrow."

print("----- BASELINE -----")
print(chat(baseline_model, baseline_tokenizer, prompt))

print("\n----- FINE-TUNED (200 steps) -----")
print(chat(ft_model, ft_tokenizer, prompt))


----- BASELINE -----
I'm feeling stressed about my exam tomorrow.You're just going to sit here and cry a lot?

----- FINE-TUNED (200 steps) -----
I'm feeling stressed about my exam tomorrow. Well, I guess I'm just going to the exam. You want to meet me by the lake this morning?


In [ ]:
# Only run this if test_ctx / test_resp are NOT already defined

def make_context_response_pairs(dialogues, max_turns=None):
    contexts = []
    responses = []
    for dialog in dialogues:
        if max_turns is not None:
            dialog = dialog[:max_turns]
        for i in range(1, len(dialog)):
            ctx = " ".join(dialog[:i])
            resp = dialog[i]
            contexts.append(ctx)
            responses.append(resp)
    return contexts, responses

test_ctx, test_resp = make_context_response_pairs(test_dialogs)

print("Num test pairs:", len(test_ctx))
print("Example context:", test_ctx[0])
print("Example response:", test_resp[0])


Num test pairs: 6740
Example context: Hey man , you wanna buy some weed ?
Example response: Some what ?


In [ ]:
import random
import torch

# Pick a small random subset for evaluation
NUM_EVAL = 50
indices = random.sample(range(len(test_ctx)), k=min(NUM_EVAL, len(test_ctx)))

eval_contexts   = [test_ctx[i] for i in indices]
eval_references = [test_resp[i] for i in indices]

def batch_generate(model, tokenizer, contexts, max_new_tokens=60):
    model.eval()
    outputs = []
    for ctx in contexts:
        inputs = tokenizer.encode(ctx + tokenizer.eos_token, return_tensors="pt")
        with torch.no_grad():
            out_ids = model.generate(
                inputs,
                max_new_tokens=max_new_tokens,   # <-- FIX
                pad_token_id=tokenizer.eos_token_id,
                do_sample=True,
                top_k=50,
                top_p=0.95,
                no_repeat_ngram_size=3,
            )
        text = tokenizer.decode(out_ids[0], skip_special_tokens=True)
        outputs.append(text)
    return outputs


print("Generating baseline outputs...")
baseline_outputs = batch_generate(baseline_model, baseline_tokenizer, eval_contexts)

print("Generating fine-tuned (200 steps) outputs...")
ft_outputs = batch_generate(ft_model, ft_tokenizer, eval_contexts)


Generating baseline outputs...
Generating fine-tuned (200 steps) outputs...


In [ ]:
print("Generating baseline outputs...")
baseline_outputs = batch_generate(baseline_model, baseline_tokenizer, eval_contexts)

print("Generating fine-tuned (200 steps) outputs...")
ft_outputs = batch_generate(ft_model, ft_tokenizer, eval_contexts)


Generating baseline outputs...
Generating fine-tuned (200 steps) outputs...


In [ ]:
import evaluate

metric_bleu  = evaluate.load("sacrebleu")
metric_rouge = evaluate.load("rouge")

def compute_metrics(preds, refs):
    # BLEU
    bleu = metric_bleu.compute(
        predictions=preds,
        references=[[r] for r in refs]
    )

    # ROUGE
    rouge = metric_rouge.compute(
        predictions=preds,
        references=refs,
        use_stemmer=True,
    )

    # Distinct-1 / Distinct-2
    def distinct_n(preds, n):
        all_ngrams = []
        for p in preds:
            toks = p.split()
            ngrams = list(zip(*[toks[i:] for i in range(n)]))
            all_ngrams.extend(ngrams)
        if not all_ngrams:
            return 0.0
        return len(set(all_ngrams)) / len(all_ngrams)

    d1 = distinct_n(preds, 1)
    d2 = distinct_n(preds, 2)

    return {
        "bleu": bleu["score"],
        "rouge1": rouge["rouge1"],
        "rouge2": rouge["rouge2"],
        "rougeL": rouge["rougeL"],
        "distinct1": d1,
        "distinct2": d2,
    }

print("Baseline metrics:")
baseline_metrics = compute_metrics(baseline_outputs, eval_references)
print(baseline_metrics)

print("\nFine-tuned (200 steps) metrics:")
ft_metrics = compute_metrics(ft_outputs, eval_references)
print(ft_metrics)


Baseline metrics:
{'bleu': 0.9164348163622075, 'rouge1': np.float64(0.12961862232111782), 'rouge2': np.float64(0.019153155149349795), 'rougeL': np.float64(0.08823635807794633), 'distinct1': 0.3980318968442484, 'distinct2': 0.8577839143942009}

Fine-tuned (200 steps) metrics:
{'bleu': 0.9013367544393213, 'rouge1': np.float64(0.14204041573140486), 'rouge2': np.float64(0.017829531846372832), 'rougeL': np.float64(0.10081258482342519), 'distinct1': 0.3824236182629592, 'distinct2': 0.8386308068459658}


In [ ]:
for i in range(3):
    print("=" * 80)
    print(f"Example {i+1}")
    print("CONTEXT:  ", eval_contexts[i])
    print("REFERENCE:", eval_references[i])
    print("BASELINE: ", baseline_outputs[i])
    print("FINE-TUNED:", ft_outputs[i])


Example 1
CONTEXT:   Please help yourself to whatever you like , don't be shy . Yes , thank you . I've already been helping myself . This dish taste terrific . Would you like to try a little ? It is a little hot , but very good . I like hot food , especially Sichuan cuisine . Would you like another beer ?
REFERENCE: I'll have another cup of beer insist . I know I don't like to drink alone , especially there are someone sitting there next to me .
BASELINE:  Please help yourself to whatever you like, don't be shy. Yes, thank you. I've already been helping myself. This dish taste terrific. Would you like to try a little? It is a little hot, but very good. I like hot food, especially Sichuan cuisine. Would you like another beer?How many hot food places would you recommend for the area?
FINE-TUNED: Please help yourself to whatever you like, don't be shy. Yes, thank you. I've already been helping myself. This dish taste terrific. Would you like to try a little? It is a little hot, but very g

In [ ]:
baseline_metrics
ft_metrics


{'bleu': 0.9013367544393213,
 'rouge1': np.float64(0.14204041573140486),
 'rouge2': np.float64(0.017829531846372832),
 'rougeL': np.float64(0.10081258482342519),
 'distinct1': 0.3824236182629592,
 'distinct2': 0.8386308068459658}

In [ ]:
ft_outputs = batch_generate(ft_model, ft_tokenizer, eval_contexts)


In [ ]:
ft_metrics = compute_metrics(ft_outputs, eval_references)
print(ft_metrics)


{'bleu': 0.9576078220639168, 'rouge1': np.float64(0.1358462065743325), 'rouge2': np.float64(0.019433654887065503), 'rougeL': np.float64(0.09223691956029148), 'distinct1': 0.3873239436619718, 'distinct2': 0.8476702508960573}
